In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os

# Load & preprocess mutation data

In [2]:
df_brca = pd.read_csv("data/tcga_brca_mutations_by_gene.csv", index_col=0)

In [3]:
df_brca.head()

,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADACL2,AADACL4,AAGAB,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-A1-A0SB-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A1-A0SD-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A1-A0SE-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A1-A0SF-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A1-A0SG-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Mutation data is {0, 1}, we don't need 64 bits
df = df_brca.astype('int8')

# Prepare cathegories

We want to know whether a sample belongs to a tumor or normal tissue.

TCGA documentation (https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/) mentions that the information is encoded in the 'BarCode'

For instance, sample `TCGA-05-4244-01A-01R-1107-07`, the fourth identifier is `01A` which means 'Tumor' (`01`), whereas sample `TCGA-91-6829-11A-01R-1858-07`, has the fourth identifier `11A` which means Normal.



In [5]:
def to_tumor_normal(barcode):
    return barcode.split('-')[3][0] == '0'

tn = [to_tumor_normal(b) for b in df.index.values]
y = np.array(tn)

# Check that we only have tumor data, not normal tissue
sum(y), sum(1-y)

(982, 0)

### Load clinical table

In [6]:
df_clinical = pd.read_csv("data/tcga_brca_clinical_data.tsv", sep="\t", low_memory=False)

In [7]:
df_clinical

,Study ID,Patient ID,Sample ID,Diagnosis Age,American Joint Committee on Cancer Metastasis Stage Code,Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code,Neoplasm Disease Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Publication Version Type,American Joint Committee on Cancer Tumor Stage Code,Brachytherapy first reference point administered total dose,...,Staging System.1,Surgery for positive margins,Surgery for positive margins other,Surgical procedure first,Time between clamping and freezing,Time between excision and freezing,Tissue Source Site,Person Neoplasm Status,Vial number,Patient's Vital Status
0,brca_tcga,TCGA-3C-AAAU,TCGA-3C-AAAU-01,55.0,MX,NX,Stage X,6th,TX,NaN,...,NaN,NaN,NaN,Modified Radical Mastectomy,NaN,NaN,3C,WITH TUMOR,A,Alive
1,brca_tcga,TCGA-3C-AALI,TCGA-3C-AALI-01,50.0,M0,N1a,Stage IIB,6th,T2,NaN,...,NaN,NaN,NaN,Lumpectomy,NaN,NaN,3C,TUMOR FREE,A,Alive
2,brca_tcga,TCGA-3C-AALJ,TCGA-3C-AALJ-01,62.0,M0,N1a,Stage IIB,7th,T2,NaN,...,NaN,NaN,NaN,Modified Radical Mastectomy,NaN,NaN,3C,TUMOR FREE,A,Alive
3,brca_tcga,TCGA-3C-AALK,TCGA-3C-AALK-01,52.0,M0,N0 (i+),Stage IA,7th,T1c,NaN,...,NaN,NaN,NaN,Simple Mastectomy,NaN,NaN,3C,TUMOR FREE,A,Alive
4,brca_tcga,TCGA-4H-AAAK,TCGA-4H-AAAK-01,50.0,M0,N2a,Stage IIIA,7th,T2,NaN,...,NaN,NaN,NaN,Modified Radical Mastectomy,NaN,NaN,4H,TUMOR FREE,A,Alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,brca_tcga,TCGA-WT-AB44,TCGA-WT-AB44-01,77.0,MX,N0 (i-),Stage IA,7th,T1c,NaN,...,NaN,NaN,NaN,Lumpectomy,NaN,NaN,WT,TUMOR FREE,A,Alive
1104,brca_tcga,TCGA-XX-A899,TCGA-XX-A899-01,46.0,MX,N2a,Stage IIIA,7th,T1c,NaN,...,NaN,NaN,NaN,Modified Radical Mastectomy,NaN,NaN,XX,TUMOR FREE,A,Alive
1105,brca_tcga,TCGA-XX-A89A,TCGA-XX-A89A-01,68.0,MX,N0,Stage IIB,7th,T3,NaN,...,NaN,NaN,NaN,Simple Mastectomy,NaN,NaN,XX,TUMOR FREE,A,Alive
1106,brca_tcga,TCGA-Z7-A8R5,TCGA-Z7-A8R5-01,61.0,MX,N1a,Stage IIIA,6th,T3,NaN,...,NaN,NaN,NaN,Other,NaN,NaN,Z7,TUMOR FREE,A,Alive


In [8]:
# Rename some columns for easier access
cli = df_clinical.rename(columns={'Sample ID': 'sample_id', 
                         'Overall Survival (Months)': 'os', 
                         'Overall Survival Status': 'os_status'}
                        )

# Drop all other columns, we are not using them
cli = cli[['sample_id', 'os', 'os_status']].copy()
cli.set_index(cli.sample_id, inplace=True)
cli.drop(columns='sample_id', inplace=True)

In [9]:
# Remove samples with missing values
to_remove = pd.isna(cli.os) | pd.isna(cli.os_status)
sum(to_remove)

6

In [10]:
cli = cli.loc[~to_remove].copy()

In [11]:
# How many in each 'Survival state'
cli.os_status.value_counts()

LIVING      948
DECEASED    154
Name: os_status, dtype: int64

In [12]:
# Censor data
cli = cli.loc[cli.os_status == 'DECEASED'].copy()

In [13]:
cli = cli[['os']].copy()
cli

,os
sample_id,
TCGA-A1-A0SK-01,31.77
TCGA-A2-A04P-01,18.00
TCGA-A2-A04V-01,63.07
TCGA-A2-A0CM-01,24.77
TCGA-A2-A0CO-01,114.72
...,...
TCGA-LL-A73Z-01,7.46
TCGA-OL-A5D6-01,36.27
TCGA-OL-A66K-01,41.89


### Intersect clinical and mutataion data

In [14]:
df = cli.join(df, how='inner').copy()

In [15]:
df.head()

,os,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADACL2,AADACL4,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-A1-A0SK-01,31.77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A04P-01,18.00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A04V-01,63.07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A0CM-01,24.77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A0CO-01,114.72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Are there samples with mutataions?
count_mut_per_sample = df.sum(axis=1)
(count_mut_per_sample == 0).sum()

0

In [17]:
# Are there genes with zero or low number of mutataions?
count_mut_per_gene = df.sum(axis=0)
keep = count_mut_per_gene > 3
keep.sum()

156

In [18]:
# Only keep genes with 3 or more mutataions
keep_names = count_mut_per_gene[keep].index
df = df[keep_names].copy()
df.head()

,os,ABCA13,ABCA8,ABCC8,ADRBK1,AKAP9,ALMS1,ANAPC1,ANK3,APC,...,UNC13C,USO1,USP34,USP9X,XIRP2,ZC3H13,ZCCHC6,ZNF208,ZNF469,ZZEF1
TCGA-A1-A0SK-01,31.77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
TCGA-A2-A04P-01,18.00,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A04V-01,63.07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A0CM-01,24.77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A0CO-01,114.72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df.sum(axis=0).min()

4.0

# Create dataset for model trainig

In [20]:
x = df.iloc[:,1:].to_numpy()
x

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [21]:
y = df.iloc[:,0].to_numpy()
y

array([ 31.77,  18.  ,  63.07,  24.77, 114.72,  77.14,   5.19,  61.89,
        27.1 ,  44.84,   8.38,  23.75,  33.9 ,  29.96,  47.27,  35.91,
        20.17,  29.01,  30.98,  37.02,   9.99, 111.99,  13.99,   5.72,
        86.6 ,  26.02,  14.65,   6.47,  81.57,  95.63,  58.9 ,  26.02,
         5.65,  55.81,  83.8 ,  17.21,  41.79,  94.15,  93.76,  20.24,
        54.17,  10.58,  77.56,  32.56,  24.61,  11.89, 129.47,  50.66,
        65.47, 146.39, 112.29,  18.76, 127.23,  84.53, 128.98, 212.09,
       102.69, 140.18,  97.4 ,  79.4 ,  18.33,  20.99, 113.7 , 129.6 ,
       244.91,  28.25,  68.89,  58.51,  37.84,  18.82,  43.5 ,  75.43,
        46.35,  32.56,  20.11,  90.77,  26.64,  72.5 ,  37.71,  30.26,
        55.58,  37.52,  66.  ,   7.36,  51.35,  51.12,  51.12,  69.88,
        91.92, 113.73, 113.73,  82.79,  42.25,  11.99,  55.65,  49.54,
        17.67,  25.79,  31.5 ,  89.09,  97.4 ,  25.07, 114.06,  33.15,
        74.67, 122.73,  33.97,  63.3 ,  54.96,  45.6 ,  72.01,  53.94,
      

Make sure the dimentions for `x` and `y` match

In [22]:
# Create list of genes
genes = list(df.columns[1:])
x.shape, y.shape, len(genes)

((140, 155), (140,), 155)

# Create Regressors and anlyze feature importance

In [23]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV

def fit_cv(model):
    ' Find best number of estimators for the model '
    param_grid = [{'n_estimators': [1, 3, 5, 10, 20, 30, 40, 50, 60, 80, 90, 100]}]
    gs = GridSearchCV(model, param_grid, cv=5)
    gs.fit(x, y)
    n = gs.best_params_['n_estimators']
    print(f"Best 'n_estimators'= {n}")
    return n

def importance(model):
    ' Show most important genes '
    model.fit(x,y)
    top = pd.Series(model.feature_importances_, genes).sort_values(ascending=False)
    print('Top genes:')
    print(top.head(10))

In [24]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)
n = fit_cv(rf)

rf = RandomForestRegressor(n_estimators=n, n_jobs=-1, random_state=42)
importance(rf)

Best 'n_estimators'= 100
Top genes:
TP53       0.055408
C5orf42    0.053313
SI         0.046321
OR5AK2     0.039489
MAP2K4     0.034253
OLFML2A    0.030662
PIK3CA     0.029774
MST1P9     0.025226
CAD        0.024349
MYH7       0.017562
dtype: float64
